In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/pii-detection-removal-from-educational-data/sample_submission.csv
/kaggle/input/pii-detection-removal-from-educational-data/train.json
/kaggle/input/pii-detection-removal-from-educational-data/test.json
/kaggle/input/gemma/transformers/7b-it/1/model.safetensors.index.json
/kaggle/input/gemma/transformers/7b-it/1/model-00003-of-00004.safetensors
/kaggle/input/gemma/transformers/7b-it/1/config.json
/kaggle/input/gemma/transformers/7b-it/1/gemma-7b-it.gguf
/kaggle/input/gemma/transformers/7b-it/1/model-00001-of-00004.safetensors
/kaggle/input/gemma/transformers/7b-it/1/tokenizer.json
/kaggle/input/gemma/transformers/7b-it/1/tokenizer_config.json
/kaggle/input/gemma/transformers/7b-it/1/model-00004-of-00004.safetensors
/kaggle/input/gemma/transformers/7b-it/1/special_tokens_map.json
/kaggle/input/gemma/transformers/7b-it/1/.gitattributes
/kaggle/input/gemma/transformers/7b-it/1/model-00002-of-00004.safetensors
/kaggle/input/gemma/transformers/7b-it/1/tokenizer.model
/kaggle/i

# **Essay generation Using Gemma**

## **Install Packages**

In [2]:
!pip install faker --quiet

In [4]:
import torch
DEVICE = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f"Device: {DEVICE}")
print(f"CUDA Version: {torch.version.cuda}")
print(f"Pytorch {torch.__version__}")

Device: cuda
CUDA Version: 12.1
Pytorch 2.1.2


In [3]:
import sys, random, string, re, time, os
import pandas as pd
import numpy as np
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer, pipeline
from tqdm.auto import tqdm
from faker import Faker  #generates fake data 
from spacy.lang.en import English

In [5]:
import ctypes, gc, torch
libc = ctypes.CDLL("libc.so.6")
def clear_memory():
    libc.malloc_trim(0)
    torch.cuda.empty_cache()
    gc.collect()

 ### **Generate fake User_id**

In [6]:
import torch, random, string

Faker.seed(0)
torch.manual_seed(42)
random.seed(42)

def get_userid(length=16):
    """Generate userid - """
    userid = str(int(np.random.rand()*1_000_000_000))
    # Add the extra rand chars
    for i in range(length):
        # Select random char or digital number (0-9)
        userid = userid + random.choice(string.ascii_letters + str(random.randint(0, 9))) 
    return userid

### **Function for generation fake social media account**

In [7]:
# Generate the personal url from social media 
def generate_fake_social_media_url(user_name):
    social_media_platforms = {
        'LinkedIn': 'linkedin.com/in/',
        'YouTube': 'youtube.com/c/',
        'Instagram': 'instagram.com/',
        'GitHub': 'github.com/',
        'Facebook': 'facebook.com/',
        'Twitter': 'twitter.com/'
    }
    platform, domain = random.choice(list(social_media_platforms.items()))
    fake_url = f'https://{domain}{user_name}'
    return fake_url


### **Generate Fake User name**

In [8]:

def generate_username(first_name, last_name, fake_user_name):
    """usernames are created from first_name and last_name"""
    SEPS = ["_", ".", ""]    
    if random.choice([False, True]):
        username = f"{first_name.lower()}{random.choice(SEPS)}{last_name.lower()}{random.randint(1,999)}"
    else:
        username = fake_user_name
    return username


### **Creating fake email id**

In [9]:
def generate_email(first_name, last_name, faker):
    """usernames are created from first_name and last_name"""
    SEPS = ["_", ".", ""]
    if random.choice([False, True]):
        email = f"{first_name.lower()}{random.choice(SEPS)}{last_name.lower()}@{faker.domain_name()}"
    else:
        email = faker.email()
    return email


### **Generate Student Information**

In [10]:

def generate_student_info():
    """Generates all the user info (name, eamil addresses, phone number, etc) together """
    # Select the student country to generate the user info based on the country
    COUNTRIES = ["en_US", "en_US", "en_US", "en_US", "en_US", "en_US",
                 "de_DE", "it_IT", "es_ES", "da_DK", "fr_FR", "vi_VN", "en_IN", "en_GB"]
    country = random.choice(COUNTRIES)
    faker = Faker(country)
    first_name = faker.first_name()
    last_name = faker.last_name()
    user_name = generate_username(first_name, last_name, faker.user_name())
    fake_url = generate_fake_social_media_url(user_name)
    student = {}
    student['COUNTRY'] = country
    student['ID_NUM'] = get_userid() # User ID
    student['NAME_STUDENT'] = first_name + " "+  last_name 
    student['EMAIL'] = generate_email(first_name, last_name, faker)
    student['USERNAME'] = user_name
    student['PHONE_NUM'] = faker.phone_number().replace(" ", "")
    student['URL_PERSONAL'] = fake_url
    student['STREET_ADDRESS'] = str(faker.address()).replace("\n"," ") # Replace \n with space in the address
    del faker
    clear_memory()
#     print(student)
    return student  

In [11]:
No_students=500
students=[]
for i in range(No_students):
    students.append(generate_student_info())
    print(f"Generated {i}-th Student information")

Generated 0-th Student information
Generated 1-th Student information
Generated 2-th Student information
Generated 3-th Student information
Generated 4-th Student information
Generated 5-th Student information
Generated 6-th Student information
Generated 7-th Student information
Generated 8-th Student information
Generated 9-th Student information
Generated 10-th Student information
Generated 11-th Student information
Generated 12-th Student information
Generated 13-th Student information
Generated 14-th Student information
Generated 15-th Student information
Generated 16-th Student information
Generated 17-th Student information
Generated 18-th Student information
Generated 19-th Student information
Generated 20-th Student information
Generated 21-th Student information
Generated 22-th Student information
Generated 23-th Student information
Generated 24-th Student information
Generated 25-th Student information
Generated 26-th Student information
Generated 27-th Student information
Ge

In [12]:
df = pd.DataFrame(students)
df.head(10)

,COUNTRY,ID_NUM,NAME_STUDENT,EMAIL,USERNAME,PHONE_NUM,URL_PERSONAL,STREET_ADDRESS
0,fr_FR,497774383piRfBbnGbmACrWSv,Camille Collin,camille_collin@lebon.org,qfournier,0492938242,https://linkedin.com/in/qfournier,"38, rue Nathalie Legrand 87438 VaillantBourg"
1,en_US,97408931wMZUIyJOxmcXfgrO,Michael Stewart,michael_stewart@green.com,michael_stewart95,560-597-5351,https://github.com/michael_stewart95,"711 Golden Overpass West Andreaville, OH 44115"
2,en_US,836848712MIkyOoXoZzeKnzji,Lisa Fernandez,barbara42@example.org,lisa_fernandez687,(209)347-1122,https://instagram.com/lisa_fernandez687,"PSC 8684, Box 8339 APO AA 76482"
3,vi_VN,427539691zoGfhOYMyMHJRRWX,Joshua Vasquez,joshua_vasquez@matthews-rogers.com,joshuavasquez599,204.313.5256,https://github.com/joshuavasquez599,"98910 Cynthia Rest South Christopherville, AZ ..."
4,en_US,693490166GGOOMjWIaubxppfU,Andrew Woodward,andrew_woodward@owen.biz,andrew_woodward465,+1-914-213-1456x208,https://linkedin.com/in/andrew_woodward465,"45792 Tammy Centers Apt. 258 Davidmouth, TN 23414"
5,en_IN,526505285JqMnWTzCCpebJLaT,Abram Keer,nehmat56@example.net,kyogi,02807150842,https://youtube.com/c/kyogi,H.No. 94 Rajagopal Path Tirupati-610935
6,en_US,239000602epQniKpZmgwAURdU,Ernest Walker,ernest_walker@baxter.org,rgray,(902)771-4278x78900,https://linkedin.com/in/rgray,63812 Richard Springs Suite 503 Lake Karenhave...
7,en_US,137966864CBrpCgPaWkFndyyY,James Franklin,james.franklin@garcia.com,morganramirez,4476142851,https://youtube.com/c/morganramirez,"34855 Meghan Squares Josephmouth, SC 57575"
8,de_DE,229699710dVddLGkGlMRzKLcf,Pero Renner,perorenner@hornich.com,pero_renner195,+49(0)2455515900,https://instagram.com/pero_renner195,Cordula-Hoffmann-Ring 8/2 17304 Parchim
9,da_DK,915526023pzQDWaNgIGiexkIN,Lea Petersen,golsen@example.net,lea.petersen210,75517176,https://twitter.com/lea.petersen210,Kielvej 729 4113 Skælskør


### **Save to csv**

In [13]:
df.to_csv("/kaggle/working/students.csv", index=False, encoding='UTF-8') 

In [125]:
student_df=pd.read_csv("students.csv")
student_df

,COUNTRY,ID_NUM,NAME_STUDENT,EMAIL,USERNAME,PHONE_NUM,URL_PERSONAL,STREET_ADDRESS
0,fr_FR,497774383piRfBbnGbmACrWSv,Camille Collin,camille_collin@lebon.org,qfournier,0492938242,https://linkedin.com/in/qfournier,"38, rue Nathalie Legrand 87438 VaillantBourg"
1,en_US,97408931wMZUIyJOxmcXfgrO,Michael Stewart,michael_stewart@green.com,michael_stewart95,560-597-5351,https://github.com/michael_stewart95,"711 Golden Overpass West Andreaville, OH 44115"
2,en_US,836848712MIkyOoXoZzeKnzji,Lisa Fernandez,barbara42@example.org,lisa_fernandez687,(209)347-1122,https://instagram.com/lisa_fernandez687,"PSC 8684, Box 8339 APO AA 76482"
3,vi_VN,427539691zoGfhOYMyMHJRRWX,Joshua Vasquez,joshua_vasquez@matthews-rogers.com,joshuavasquez599,204.313.5256,https://github.com/joshuavasquez599,"98910 Cynthia Rest South Christopherville, AZ ..."
4,en_US,693490166GGOOMjWIaubxppfU,Andrew Woodward,andrew_woodward@owen.biz,andrew_woodward465,+1-914-213-1456x208,https://linkedin.com/in/andrew_woodward465,"45792 Tammy Centers Apt. 258 Davidmouth, TN 23414"
...,...,...,...,...,...,...,...,...
495,en_US,9107118914I1bbhleCBQatynB,Kathryn Smith,kathrynsmith@delacruz-nelson.com,kathryn_smith774,(388)814-4949x30998,https://github.com/kathryn_smith774,"90006 Vanessa Road Suite 480 Whitetown, NC 98188"
496,da_DK,715305063cubSZWXBNRbcWEIf,Gunnar Lassen,victoria17@example.net,zahra84,74454055,https://twitter.com/zahra84,Stokrosegade 22 3891 Blåvand
497,vi_VN,511792705VpEuJnAaLGelryPq,Jason Hill,sarah77@example.com,paul11,348.238.8235x030,https://github.com/paul11,"9623 Justin Crossing Suite 244 Millerfurt, NV ..."
498,vi_VN,834695186ksvheEpyXclvFTZg,Stephen Romero,leslie78@example.com,stephen_romero924,+1-684-482-5873,https://github.com/stephen_romero924,"7781 Escobar Fords Apt. 381 New Mackenzieview,..."


### **Generate texts Using Gemma Models**

# **Load Gemma models**

In [13]:
import pandas as pd
import numpy as np
from transformers import (BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer, pipeline)
from tqdm.auto import tqdm

In [14]:
import torch, random
# Ensure that all operations are deterministic on GPU (if used) for reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

SEED = 42
# Seed the same seed to all 
def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

seed_everything(SEED)

In [15]:
import ctypes, gc, torch
libc = ctypes.CDLL("libc.so.6")
def clear_memory():
    libc.malloc_trim(0)
    torch.cuda.empty_cache()
    gc.collect()

In [16]:
model_path="/kaggle/input/gemma/transformers/7b-it/1"
# Load the model in 4-bit precision and float16 type
compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
)
# Load the model
model = AutoModelForCausalLM.from_pretrained(model_path,
                                             device_map="auto",
                                             quantization_config=bnb_config)
model.config.use_cache = False
model.config.pretraining_tp = 1
tokenizer = AutoTokenizer.from_pretrained(model_path)

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

### **Create prompt and generate Essay for all students**

In [423]:
def create_prompt(row, columns):
    # Prompt template generates the texts
    prompt_template = """
    You are an essay writer. 
    You will be given some personal information.
    Given information has the following keys:

    NAME_STUDENT - The full or partial name of a student that is not necessarily the author of the essay. This excludes instructors, authors, and other person names.
    EMAIL - A student’s email address.
    USERNAME - A student's username on any platform.
    ID_NUM - A number or sequence of characters that could be used to identify a student, such as a student ID or a social security number.
    PHONE_NUM - A phone number associated with a student.
    URL_PERSONAL - A URL that might be used to identify a student.
    STREET_ADDRESS - A full or partial street address that is associated with the student, such as their home address.

    Each of these will have a list of values associated with them. 
        
    {input_str}
    
    You need to write a short essay about experiences of this person in either technology or tourist places or space including atleast 5 information
    of a person such as NAME_STUDENT,EMAIL,USERNAME,ID_NUM,PHONE_NUM,URL_PERSONAL,STREET_ADDRESS. """
    
    # Collect student inform
    inputs = []
    for column in columns:
        inputs.append(f"{column} = {row[column]}")
    input_str = "\n".join(inputs)
    # fill input to prompt
    prompt = prompt_template.format(input_str=input_str)
    return prompt

def generate_texts(model, tokenizer, df, start_pos, end_pos):
    generated_df = df[start_pos:end_pos]
    columns = generated_df.columns
    generated_df = generated_df.reset_index(drop=True)
    generated_texts = []
    # Generate the texts
    for i in tqdm(range(len(generated_df))):
        start = time.time()
        row = generated_df.loc[i]
        try:
            # Generate the texts using the prompts
            prompt = create_prompt(row, columns)
            # print(f"prompt = {prompt}")
            # Tokenize the prompt
            inputs = tokenizer(prompt, return_tensors="pt").to(DEVICE)
            # Generate the outputs from prompt
            generate_ids = model.generate(**inputs, max_new_tokens=768)
            # Decode the generated output
            generated_text = tokenizer.batch_decode(generate_ids, skip_special_tokens=True,
                                                     clean_up_tokenization_spaces=False)[0]
            if '**Essay:**' in generated_text:
                generated_text = generated_text.split("**Essay:**")[1].strip()
            else:
                generated_text = "Unprocessed: " + generated_text
            generated_texts.append(generated_text)
            #print(f"generated_text = {generated_text}" )
        except Exception as e:
            print(f"Error occurs during text generate {e}")
            generated_texts.append("Errors")
        clear_memory()
        print(f"Completed writing essay for {i}-th student {time.time() - start: .1f} seconds")
    assert len(generated_df) == len(generated_texts), "Generate texts length is mismatched"
    # Store all the generated texts
    generated_df['generated_text'] = generated_texts
    # Save generated_df to csv
    generated_df.to_csv(f"generated_df_{start_pos}_{end_pos}.csv", index=False, encoding="UTF-8")

In [424]:
num_essays = 500
start_pos = 0
end_pos = start_pos + num_essays

# Generate the texts
generate_texts(model, tokenizer, student_df, start_pos, end_pos)

  0%|          | 0/500 [00:00<?, ?it/s]

Completed writing essay for 0-th student  31.9 seconds
Completed writing essay for 1-th student  25.5 seconds
Completed writing essay for 2-th student  30.7 seconds
Completed writing essay for 3-th student  25.5 seconds
Completed writing essay for 4-th student  30.2 seconds
Completed writing essay for 5-th student  27.3 seconds
Completed writing essay for 6-th student  29.1 seconds
Completed writing essay for 7-th student  26.0 seconds
Completed writing essay for 8-th student  27.5 seconds
Completed writing essay for 9-th student  24.2 seconds
Completed writing essay for 10-th student  23.6 seconds
Completed writing essay for 11-th student  29.7 seconds
Completed writing essay for 12-th student  25.2 seconds
Completed writing essay for 13-th student  27.2 seconds
Completed writing essay for 14-th student  30.4 seconds
Completed writing essay for 15-th student  27.6 seconds
Completed writing essay for 16-th student  28.5 seconds
Completed writing essay for 17-th student  30.8 seconds
Co

In [425]:
generated_df=pd.read_csv("generated_df_0_500.csv")
generated_df = generated_df.rename({'generated_text': 'Essay'}, axis=1)
generated_df

,COUNTRY,ID_NUM,NAME_STUDENT,EMAIL,USERNAME,PHONE_NUM,URL_PERSONAL,STREET_ADDRESS,Essay
0,fr_FR,497774383piRfBbnGbmACrWSv,Camille Collin,camille_collin@lebon.org,qfournier,0492938242,https://linkedin.com/in/qfournier,"38, rue Nathalie Legrand 87438 VaillantBourg",Unprocessed: \n You are an essay writer. \n...
1,en_US,97408931wMZUIyJOxmcXfgrO,Michael Stewart,michael_stewart@green.com,michael_stewart95,560-597-5351,https://github.com/michael_stewart95,"711 Golden Overpass West Andreaville, OH 44115",Unprocessed: \n You are an essay writer. \n...
2,en_US,836848712MIkyOoXoZzeKnzji,Lisa Fernandez,barbara42@example.org,lisa_fernandez687,(209)347-1122,https://instagram.com/lisa_fernandez687,"PSC 8684, Box 8339 APO AA 76482",Unprocessed: \n You are an essay writer. \n...
3,vi_VN,427539691zoGfhOYMyMHJRRWX,Joshua Vasquez,joshua_vasquez@matthews-rogers.com,joshuavasquez599,204.313.5256,https://github.com/joshuavasquez599,"98910 Cynthia Rest South Christopherville, AZ ...",Unprocessed: \n You are an essay writer. \n...
4,en_US,693490166GGOOMjWIaubxppfU,Andrew Woodward,andrew_woodward@owen.biz,andrew_woodward465,+1-914-213-1456x208,https://linkedin.com/in/andrew_woodward465,"45792 Tammy Centers Apt. 258 Davidmouth, TN 23414",Unprocessed: \n You are an essay writer. \n...
...,...,...,...,...,...,...,...,...,...
495,en_US,9107118914I1bbhleCBQatynB,Kathryn Smith,kathrynsmith@delacruz-nelson.com,kathryn_smith774,(388)814-4949x30998,https://github.com/kathryn_smith774,"90006 Vanessa Road Suite 480 Whitetown, NC 98188",Unprocessed: \n You are an essay writer. \n...
496,da_DK,715305063cubSZWXBNRbcWEIf,Gunnar Lassen,victoria17@example.net,zahra84,74454055,https://twitter.com/zahra84,Stokrosegade 22 3891 Blåvand,Unprocessed: \n You are an essay writer. \n...
497,vi_VN,511792705VpEuJnAaLGelryPq,Jason Hill,sarah77@example.com,paul11,348.238.8235x030,https://github.com/paul11,"9623 Justin Crossing Suite 244 Millerfurt, NV ...",Unprocessed: \n You are an essay writer. \n...
498,vi_VN,834695186ksvheEpyXclvFTZg,Stephen Romero,leslie78@example.com,stephen_romero924,+1-684-482-5873,https://github.com/stephen_romero924,"7781 Escobar Fords Apt. 381 New Mackenzieview,...",Unprocessed: \n You are an essay writer. \n...


### **Convert Generated texts into competition format**

In [426]:
label_types = ['NAME_STUDENT','EMAIL', 'USERNAME', 'ID_NUM',
               'PHONE_NUM', 'URL_PERSONAL', 'STREET_ADDRESS']

Label assignment functions

In [427]:
from spacy.lang.en import English
import re

en_tokenizer = English().tokenizer
    
def tokenize_with_spacy(text, tokenizer=en_tokenizer):
    tokenized_text = tokenizer(text)
    tokens = [token.text for token in tokenized_text]
    trailing_whitespace = [bool(token.whitespace_) for token in tokenized_text]
    return tokens, trailing_whitespace

# Update labels and boolean flags
def update_labels(i, token, label_type, labels, isFirst_flags):
#     print(f"Found {i}-th position token: {token}")
    # Update the label
    if isFirst_flags[label_type]:
        labels[i] = 'B-'+label_type # Beginning of an entity
        isFirst_flags[label_type] = False
    else:
        labels[i] = 'I-'+label_type # Contiunity of an entity
    return labels, isFirst_flags

# Go through each token and assign name label ('NAME_STUDENT') if matched
def assign_name(names, tokens, labels):
    #print(f"Search 'NAME_STUDENT': {names}")
    for i, token in enumerate(tokens):
        token = str(token).lower()
        # Order does not matter
        if token in names:
            # If the previous 
            if i > 0 and labels[i-1] == 'B-NAME_STUDENT':
                labels[i] = 'I-NAME_STUDENT'
            else:
                labels[i] = 'B-NAME_STUDENT'        
    return labels


In [428]:
def assign_phone_number(phones, tokens, labels, isFirst_flags):
#     print(f"Search 'PHONE_NUM': {phone_tokens}")
    for i, token in enumerate(tokens):
        token_ = str(token).lower()
        label = labels[i]
        # Add a special case '-'
        if token == '-':
            # Check if the previous token is not 'O'
            if labels[i-1] != 'O':
                # Update the labels
                labels, isFirst_flags = update_labels(i, token_, 'PHONE_NUM',
                                                      labels, isFirst_flags)
        else:
            # Check if token is matched last name
            if label == 'O' and token_ in phones:
                labels, isFirst_flags = update_labels(i, token_, 'PHONE_NUM',
                                                      labels, isFirst_flags)
    return labels, isFirst_flags

In [429]:
# Go through each token and assign street address label
def assign_street_address(address, tokens, labels):
    # print(tokens)
    # Keep track of index for a long label
    label_index = 0
    reserve_indices = []
    sandwich_max_size = 3
    #print(f'address = {address}')
    for i, token in enumerate(tokens):
        try:
            #Order matters and sandwiches are possible
            token = str(token).lower()
            curr_idx = label_index
            curr_token = address[curr_idx] 
            if token == curr_token:
                # print(f"token = {token}")               
                #case where a token corresponds to the expected next token
                if len(reserve_indices) > sandwich_max_size:
                    reserve_indices = []

                prefix = 'B-' if curr_idx == 0 else 'I-'
                labels[i] = prefix + 'STREET_ADDRESS'
                #fill sandwiches if the next token has been found
                for k in reserve_indices:
                    labels[k] = 'I-STREET_ADDRESS'
                reserve_indices = []
                #Update positional pointer
                label_index += 1
                # At the end of address
                if label_index == len(address):
                    label_index = 0
                # print(f'label_index = {label_index}')
            else:
                reserve_indices.append(i)
                #print(f"! {token}")
        except Exception as e:
            print(f"Error occurs at {i}-th {token} \n{e}" )
            sys.exit(-1)

    return labels


In [430]:
# Assign labels for other types
def assign_other_label_types(essay, label_type, tokens, labels, isFirst_flags):
    label_value = essay[label_type].lower()
#     print(f"Search '{label_type}': {label_value}")    
    for i, token in enumerate(tokens):
        token_ = str(token).lower()
        # Check if token is first or last name
        if label_value in token_:
            # Update the label
            labels, isFirst_flags = update_labels(i, token_, label_type,
                                                      labels, isFirst_flags)
    return labels, isFirst_flags

# Assign labels for other types
def assign_email(email, tokens, labels):
    #print(f"Search Email: {email}")
    is_First = False
    for i, token in enumerate(tokens):
        token_ = str(token).lower()
        # Check if token is first or last name
        if email in token_:
            # print(f'Token {token_}')
            if not is_First: 
                # Update the label
                labels[i] = 'B-EMAIL'
                is_First = True
            else: # Skip labeling
                print(f"Duplicated Email {email}")
    return labels

# Assign labels for other types
def assign_username(username, tokens, labels):
    #print(f"Search username: {username}")
    is_First = False
    for i, token in enumerate(tokens):
        token_ = str(token).lower()
        # Check if token is first or last name
        if username in token_:
            # print(f'Token {token_}')
            if not is_First: 
                # Update the label
                labels[i] = 'B-USERNAME'
                is_First = True
            else: # Skip labeling
                print(f"Duplicated username {username}")
    return labels

In [431]:
# “B-”: the beginning of an entity. 
# “I-”: the next of an entity
def assign_labels(essay, tokens):
    # print(f"essay['NAME_STUDENT'] = {essay['NAME_STUDENT']}" )
    # Assign "O" to labels by default
    labels = ['O' for token in tokens] 
    # Create a boolean flag list to track if a label type start the text.
    isFirst_flags = {label_type: True for label_type in label_types}
    # Go through each token and check if the label appear in the token
    # All token and label values are lower case for comparison
    for label_type in label_types:
        if label_type == 'NAME_STUDENT':
            if essay['NAME_STUDENT'] != '':
                names, _ = tokenize_with_spacy(essay['NAME_STUDENT'].lower())
                labels = assign_name(names, tokens, labels)
        elif label_type == 'STREET_ADDRESS':
            address = essay['STREET_ADDRESS'].lower().replace("\\n", " ")
            # print(f"address {address}")
            address = address.translate(str.maketrans('', '', string.punctuation)) # Remove punctuations
            address = address.split(' ')
            if len(address) > 0:                
                labels = assign_street_address(address, tokens, labels)
        elif label_type == 'PHONE_NUM':
            phones, _ = tokenize_with_spacy(essay['PHONE_NUM'].lower())
            labels, isFirst_flags = assign_phone_number(phones, tokens, labels, isFirst_flags) 
        elif label_type == 'EMAIL':
            email = essay['EMAIL'].lower()
            assign_email(email, tokens, labels)
        elif label_type == 'USERNAME':
            username = essay['USERNAME'].lower()
            assign_username(username, tokens, labels)
        else:
            labels, isFirst_flags = assign_other_label_types(essay, label_type, tokens,
                                                            labels, isFirst_flags)
    return labels 

In [432]:
def process_full_text(full_text):    
    if 'Unprocessed:' in full_text: # Need to split the instruction and response        
        pattern = r'([*]*Please write an essay about [^*]*[*]*)'
        x = re.search(pattern, full_text)
        if x:
            splits = re.split(pattern, full_text)
            text = splits[-1].strip()
            # print(f"=== Split text ===\n {text}")
            return text
        else:
            print(f"### UnProcess text###\n {full_text}")            
            return None
    else:
        return full_text.strip() # Remove the space to the left

# Map the label to token
def create_token_map(tokens, labels):
    token_map = []
    for i, label in enumerate(labels):
        if label != 'O':
            token_map.append({label: (tokens[i], i)})
    return token_map

In [433]:
# Assign the labels to the tokens           
results = []
doc_id = 5_121_777_000 # document id 
for i in range(len(generated_df)):
    row = generated_df.iloc[i]
    full_text = process_full_text(row["Essay"])
    if full_text:
        # Tokenize the text using spacy tokenizer
        tokens, trailing_whitespace = tokenize_with_spacy(full_text)
        labels = assign_labels(row, tokens)
        token_map = create_token_map(tokens, labels)
        doc_id += 1
        result = {
            'document': doc_id, 
            'full_text': full_text,
            'tokens': tokens, 
            'trailing_whitespace': trailing_whitespace,
            'labels': labels,
            'token_map': token_map
        }
        # Add PII to result
        for label_type in label_types:
            result[label_type] = row[label_type]
        # print(result)
        results.append(result)
# # Save to temp fold for verification
df = pd.DataFrame(results)
df.to_csv("pii_dataset_using_Gemma.csv", index=False, encoding="UTF-8")
display(df)

### UnProcess text###
 Unprocessed: 
    You are an essay writer. 
    You will be given some personal information.
    Given information has the following keys:

    NAME_STUDENT - The full or partial name of a student that is not necessarily the author of the essay. This excludes instructors, authors, and other person names.
    EMAIL - A student’s email address.
    USERNAME - A student's username on any platform.
    ID_NUM - A number or sequence of characters that could be used to identify a student, such as a student ID or a social security number.
    PHONE_NUM - A phone number associated with a student.
    URL_PERSONAL - A URL that might be used to identify a student.
    STREET_ADDRESS - A full or partial street address that is associated with the student, such as their home address.

    Each of these will have a list of values associated with them. 
        
    COUNTRY = fr_FR
ID_NUM = 497774383piRfBbnGbmACrWSv
NAME_STUDENT = Camille Collin
EMAIL = camille_collin@lebon.org

,document,full_text,tokens,trailing_whitespace,labels,token_map,NAME_STUDENT,EMAIL,USERNAME,ID_NUM,PHONE_NUM,URL_PERSONAL,STREET_ADDRESS
0,5121777001,**Michael Stewart's Experiences**\n\nMichael S...,"[*, *, Michael, Stewart, 's, Experiences, *, *...","[False, False, True, False, True, False, False...","[O, O, B-NAME_STUDENT, I-NAME_STUDENT, O, O, O...","[{'B-NAME_STUDENT': ('Michael', 2)}, {'I-NAME_...",Michael Stewart,michael_stewart@green.com,michael_stewart95,97408931wMZUIyJOxmcXfgrO,560-597-5351,https://github.com/michael_stewart95,"711 Golden Overpass West Andreaville, OH 44115"
1,5121777002,**Lea Petersen's Experiences**\n\nLea Petersen...,"[*, *, Lea, Petersen, 's, Experiences, *, *, \...","[False, False, True, False, True, False, False...","[O, O, B-NAME_STUDENT, I-NAME_STUDENT, O, O, O...","[{'B-NAME_STUDENT': ('Lea', 2)}, {'I-NAME_STUD...",Lea Petersen,golsen@example.net,lea.petersen210,915526023pzQDWaNgIGiexkIN,75517176,https://twitter.com/lea.petersen210,Kielvej 729 4113 Skælskør
2,5121777003,**Vaibhav Manne's Experiences**\n\nVaibhav Man...,"[*, *, Vaibhav, Manne, 's, Experiences, *, *, ...","[False, False, True, False, True, False, False...","[O, O, B-NAME_STUDENT, I-NAME_STUDENT, O, O, O...","[{'B-NAME_STUDENT': ('Vaibhav', 2)}, {'I-NAME_...",Vaibhav Manne,vaibhav_manne@kuruvilla.com,saanvi71,604868807YyEmw4oQzrXwzvhl,+912040752275,https://instagram.com/saanvi71,"18/91, Zachariah Marg, Ramagundam-769930"
3,5121777004,**Lorenza Iriarte's Experiences**\n\nLorenza I...,"[*, *, Lorenza, Iriarte, 's, Experiences, *, *...","[False, False, True, False, True, False, False...","[O, O, B-NAME_STUDENT, I-NAME_STUDENT, O, O, O...","[{'B-NAME_STUDENT': ('Lorenza', 2)}, {'I-NAME_...",Lorenza Iriarte,lorenza.iriarte@mineria.com,ortunocalisto,555705611iQJuNBCVCWOXFrep,+34844303104,https://linkedin.com/in/ortunocalisto,"Vial Glauco Santiago 8 Teruel, 17922"
4,5121777005,"Sumer Kashyap, a student with a captivating bl...","[Sumer, Kashyap, ,, a, student, with, a, capti...","[True, False, True, True, True, True, True, Tr...","[B-NAME_STUDENT, I-NAME_STUDENT, O, O, O, O, O...","[{'B-NAME_STUDENT': ('Sumer', 0)}, {'I-NAME_ST...",Sumer Kashyap,ritvikacharya@example.net,sumer.kashyap855,992137555bdoXWpbjiQKDXkMX,09703414892,https://linkedin.com/in/sumer.kashyap855,"H.No. 738 Dey Marg, Jamshedpur-004689"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,5121777170,In the realm of technological advancements and...,"[In, the, realm, of, technological, advancemen...","[True, True, True, True, True, True, True, Tru...","[O, O, O, O, O, O, O, O, O, O, O, O, O, B-NAME...","[{'B-NAME_STUDENT': ('Julia', 13)}, {'I-NAME_S...",Julia Barlow,juliabarlow@smith.org,claire63,176861671aYjlgzjvbztrmMBk,01414960091,https://github.com/claire63,Studio 2 Kelly estate East Amandashire CM1 7WE
170,5121777171,In the realm of technological advancements and...,"[In, the, realm, of, technological, advancemen...","[True, True, True, True, True, True, True, Tru...","[O, O, O, O, O, O, O, O, O, O, B-NAME_STUDENT,...","[{'B-NAME_STUDENT': ('Rebecca', 10)}, {'I-NAME...",Rebecca Padilla,rebecca_padilla@myers.org,rebecca_padilla69,896915177MjHZAaOvRBhisUZx,001-432-599-6452x793,https://instagram.com/rebecca_padilla69,"67432 Ross Points Nicholasville, CO 04333"
171,5121777172,"In the tapestry of human experiences, technolo...","[In, the, tapestry, of, human, experiences, ,,...","[True, True, True, True, True, False, True, Fa...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[{'B-NAME_STUDENT': ('Jimmy', 39)}, {'I-NAME_S...",Jimmy Simonsen,jimmy.simonsen@schmidt.org,leonora24,285399250VHsYBtnsbfDCJMWg,17037225,https://instagram.com/leonora24,Lombardivej 5 9425 Veksø Sjælland
172,5121777173,"Merte Skov, a student with a captivating onlin...","[Merte, Skov, ,, a, student, with, a, captivat...","[True, False, True, True, True, True, True, Tr...","[O, B-NAME_STUDENT, O, O, O, O, O, O, O, O, O,...","[{'B-NAME_STUDENT': ('Skov', 1)}, {'B-USERNAME...",Merete Sk

In [434]:
pii_train_df = df[['document', 'full_text', 'tokens', 'trailing_whitespace', 'labels']]
pii_train_df.to_json("Gemma_generated_data.json")
display(pii_train_df.tail(10))

,document,full_text,tokens,trailing_whitespace,labels
164,5121777165,"Indranil Grover, a student with a captivating ...","[Indranil, Grover, ,, a, student, with, a, cap...","[True, False, True, True, True, True, True, Tr...","[B-NAME_STUDENT, I-NAME_STUDENT, O, O, O, O, O..."
165,5121777166,"In the tapestry of human experiences, Nicholas...","[In, the, tapestry, of, human, experiences, ,,...","[True, True, True, True, True, False, True, Tr...","[O, O, O, O, O, O, O, B-NAME_STUDENT, I-NAME_S..."
166,5121777167,I am unable to write an essay about Helen Dahl...,"[I, am, unable, to, write, an, essay, about, H...","[True, True, True, True, True, True, True, Tru...","[O, O, O, O, O, O, O, O, B-NAME_STUDENT, I-NAM..."
167,5121777168,In the realm of technological advancements and...,"[In, the, realm, of, technological, advancemen...","[True, True, True, True, True, True, True, Tru...","[O, O, O, O, O, O, O, O, O, O, B-NAME_STUDENT,..."
168,5121777169,**Jack Anderson's Experiences**\n\nJack Anders...,"[*, *, Jack, Anderson, 's, Experiences, *, *, ...","[False, False, True, False, True, False, False...","[O, O, B-NAME_STUDENT, I-NAME_STUDENT, O, O, O..."
169,5121777170,In the realm of technological advancements and...,"[In, the, realm, of, technological, advancemen...","[True, True, True, True, True, True, True, Tru...","[O, O, O, O, O, O, O, O, O, O, O, O, O, B-NAME..."
170,5121777171,In the realm of technological advancements and...,"[In, the, realm, of, technological, advancemen...","[True, True, True, True, True, True, True, Tru...","[O, O, O, O, O, O, O, O, O, O, B-NAME_STUDENT,..."
171,5121777172,"In the tapestry of human experiences, technolo...","[In, the, tapestry, of, human, experiences, ,,...","[True, True, True, True, True, False, True, Fa...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
172,5121777173,"Merte Skov, a student with a captivating onlin...","[Merte, Skov, ,, a, student, with, a, captivat...","[True, False, True, True, True, True, True, Tr...","[O, B-NAME_STUDENT, O, O, O, O, O, O, O, O, O,..."
173,5121777174,"In the tapestry of human experiences, Gunnar L...","[In, the, tapestry, of, human, experiences, ,,...","[True, True, True, True, True, False, True, Tr...","[O, O, O, O, O, O, O, B-NAME_STUDENT, I-NAME_S..."


In [2]:
import sys
import random
import string, re, time
import pandas as pd
import numpy as np
from transformers import (BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer, pipeline)
from tqdm.auto import tqdm

2024-05-11 22:30:08.157447: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-11 22:30:08.157546: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-11 22:30:08.299234: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
model = AutoModelForCausalLM.from_pretrained("/kaggle/input/gemma/pytorch/7b-it/2",
                                             device_map="auto",
                                             quantization_config=bnb_config)

In [ ]:
model_path="/kaggle/input/gemma/transformers/7b-it/1"
# Load the model in 4-bit precision and float16 type
compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
)

In [1]:
!pip install seqeval evaluate -q


In [16]:
import json

# Load JSON data from file
with open('/kaggle/input/pii-detection-removal-from-educational-data/train.json', 'r') as json_file:
    data = json.load(json_file)

pos=[] #positive tokens(PII)
neg=[] #negative tokens(Non-PII)

# Iterate through each data point
for d in data:
    if any(np.array(d["labels"]) != "O"):
        # If any non-"O" label is found, append to positive samples
        pos.append(d)  #PII tokens and labels
    else:
        neg.append(d)  #non PII

# Print the original number of datapoints
print(f"original datapoints: {len(data)}, PII tokens: {len(pos)}, Non PII tokens: {len(neg)}")


original datapoints: 6807, PII tokens: 945, Non PII tokens: 5862


In [17]:
len(data)

6807